In [51]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.runconfig import MpiConfiguration
from azureml.core.runconfig import RunConfiguration, DockerConfiguration


In [52]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

In [53]:
# training script
source_dir = "train"
script_name = "dist_train_bert.py"

# environment file
environment_file = os.path.join("train", "dist_train_bert_env.yml")

# azure ml settings
environment_name = "pl-env-lang"
experiment_name = "dist-bert-livedoor"
compute_name = "shuit-gpu-clus04"

In [66]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

docker_config = DockerConfiguration(use_docker=True)

env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04:20211221.v1"
)

In [77]:
# 学習設定

num_nodes = 4
num_gpus = 1

target_batch_size = 256
lr = 0.01

cluster = ws.compute_targets[compute_name]
#dist_config = PyTorchConfiguration(node_count=num_nodes)
dist_config = MpiConfiguration(node_count=num_nodes)

src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=[
        "--batch_size", target_batch_size/(num_nodes*num_gpus),
        "--learning_rate", lr,
        "--max_epochs", 40,
        "--accelerator", "gpu",
        "--gpus", num_gpus,
        "--strategy", "ddp",
        "--num_nodes", num_nodes,
    ],
    compute_target=cluster,
    environment=env,
    docker_runtime_config=docker_config,
    distributed_job_config=dist_config,
)

In [78]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

RunId: dist-bert-livedoor_1642064366_3c2ffbd7
Web View: https://ml.azure.com/runs/dist-bert-livedoor_1642064366_3c2ffbd7?wsid=/subscriptions/902f236f-44df-463a-a5cb-1516ab2a9cd2/resourcegroups/shuit-common/workspaces/shuit-ml-workspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_01347f797116d84b8079b6e9045193ede103ba5d59e24451b715f3996fa90ce3_p.txt

2022-01-13T09:02:28Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/dist-bert-livedoor_1642064366_3c2ffbd7/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/dist-bert-livedoor_1642064366_3c2ffbd7/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=692637 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/dist-bert-livedoor_1642064366_3c2ffbd7/configs/workspaceblobstore.cfg --log-level=

In [13]:
# モデル登録
run.register_model(
    model_name="bert-livedoor-model",
    model_path=os.path.join('outputs', 'model.ckpt'),
    model_framework=Model.Framework.PYTORCH,
)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.onnx in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_0f93e9792dd9ffbd80a0b0cbeb00a8fab0b14be474a32b815609bb1f4bffa494_d.txt', 'azureml-logs/55_azureml-execution-tvmps_6ad0f50cd79742bed158768f299d376ef743927df3983464d8dc993376693eb2_d.txt', 'azureml-logs/65_job_prep-tvmps_0f93e9792dd9ffbd80a0b0cbeb00a8fab0b14be474a32b815609bb1f4bffa494_d.txt', 'azureml-logs/65_job_prep-tvmps_6ad0f50cd79742bed158768f299d376ef743927df3983464d8dc993376693eb2_d.txt', 'azureml-logs/70_driver_log_0.txt', 'azureml-logs/70_driver_log_1.txt', 'azureml-logs/70_mpi_log.txt', 'azureml-logs/75_job_post-tvmps_0f93e9792dd9ffbd80a0b0cbeb00a8fab0b14be474a32b815609bb1f4bffa494_d.txt', 'azureml-logs/75_job_post-tvmps_6ad0f50cd79742bed158768f299d376ef743927df3983464d8dc993376693eb2_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azure